<a href="https://colab.research.google.com/github/alwaysneedhelp/Practice-for-IOAI/blob/main/objects_classifier_again_using_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SECOND ATTEMPT AFTER WATCHING THE TUTORIAL**

watched it twice again, just to understand what is going on

In [ ]:
import numpy as np
from PIL import Image

import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.optim as optim


import torchvision
import torchvision.transforms as trans

In [ ]:
transformed = trans.Compose([
    trans.ToTensor(),
    trans.Normalize((0.5, 0.5, 0.5), (0.5,0.5,0.5))
])

In [ ]:
train_data = torchvision.datasets.CIFAR10(root='data', download=True, train=True, transform=transformed)
test_data = torchvision.datasets.CIFAR10(root='data', download=True, train=False, transform=transformed)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

In [ ]:
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

In [ ]:
class NeuralNet(nn.Module):

  def __init__(self):
    super(NeuralNet, self).__init__()

    self.conv1 = nn.Conv2d(3, 12, 5)
    self.conv2 = nn.Conv2d(12, 24, 5)
    self.pool = nn.MaxPool2d((2,2))
    self.fc1 = nn.Linear(24*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)


  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x, 1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [ ]:
for name, param in NeuralNet().named_parameters():
    print(name, param.shape)

conv1.weight torch.Size([12, 3, 5, 5])
conv1.bias torch.Size([12])
conv2.weight torch.Size([24, 12, 5, 5])
conv2.bias torch.Size([24])
fc1.weight torch.Size([120, 600])
fc1.bias torch.Size([120])
fc2.weight torch.Size([84, 120])
fc2.bias torch.Size([84])
fc3.weight torch.Size([10, 84])
fc3.bias torch.Size([10])


In [ ]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(50):
  print(f'Training epoch {epoch}...')

  running_loss = 0.0

  for i, data in enumerate(train_loader):
    inputs, labels = data
    optimizer.zero_grad()

    outputs = net(inputs)
    loss = loss_function(outputs, labels)

    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  print(f'Loss: {(running_loss/len(train_loader)):.4f}')

Training epoch 0...
Loss: 2.1917
Training epoch 1...
Loss: 1.7520
Training epoch 2...
Loss: 1.5407
Training epoch 3...
Loss: 1.4269
Training epoch 4...
Loss: 1.3369
Training epoch 5...
Loss: 1.2508
Training epoch 6...
Loss: 1.1789
Training epoch 7...
Loss: 1.1151
Training epoch 8...
Loss: 1.0621
Training epoch 9...
Loss: 1.0147
Training epoch 10...
Loss: 0.9737
Training epoch 11...
Loss: 0.9352
Training epoch 12...
Loss: 0.8971
Training epoch 13...
Loss: 0.8648
Training epoch 14...
Loss: 0.8285
Training epoch 15...
Loss: 0.8002
Training epoch 16...
Loss: 0.7674
Training epoch 17...
Loss: 0.7375
Training epoch 18...
Loss: 0.7168
Training epoch 19...
Loss: 0.6884
Training epoch 20...
Loss: 0.6597
Training epoch 21...
Loss: 0.6349
Training epoch 22...
Loss: 0.6135
Training epoch 23...
Loss: 0.5951
Training epoch 24...
Loss: 0.5689
Training epoch 25...
Loss: 0.5443
Training epoch 26...
Loss: 0.5232
Training epoch 27...
Loss: 0.5034
Training epoch 28...
Loss: 0.4817
Training epoch 29...
Los

In [ ]:
torch.save(net.state_dict(), 'models/objects_classifier_torch.pth')

In [ ]:
net = NeuralNet()
net.load_state_dict(torch.load('models/objects_classifier_torch.pth'))

In [ ]:
correct = 0
total = 0


net.eval()

with torch.no_grad():
  for data in test_loader:
    inputs, labels = data
    outputs = net(inputs)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted==labels).sum().item()

accuracy = 100 * correct / total

print(f'Accuracy is {accuracy}')

Accuracy is 69.29


In [ ]:
new_transform = trans.Compose([
    trans.Resize((32, 32)),
    trans.ToTensor(),
    trans.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

In [ ]:
def load_image(image_path):
  image = Image.open(image_path)
  image = new_transform(image)
  image = image.unsqueeze(0)
  return image

image_paths = ['./input_data/dog.jpg', './input_data/plane.jpg']

images = [load_image(img) for img in image_paths]

with torch.no_grad():
  for image in images:
    output = net(image)
    _, predicted = torch.max(output, 1)
    print(f'Prediction: {class_names[predicted.item()]}')

Prediction: dog
Prediction: airplane
